In [31]:
#>>>>>>> 1
import pandas as pd 
import numpy as np 
import csv
from scipy.stats import ttest_rel,ttest_ind
from statsmodels.stats.multitest import multipletests

In [32]:
#>>>>>>> 2
# Reading the healthy and cancerous genes into dataframes
healthydf = pd.read_csv('healthy.csv', sep = '\t', index_col=None)
cancerousdf = pd.read_csv('cancerous.csv', sep = '\t', index_col =None)
#healthydf     # from here we knew that the dimensions are ( 19648 rows × 51 columns ) and the structure
#cancerousdf     # to make sure that gens are the same in names and id  
cancerousdf

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LIN7B,64130,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34
2,LXN,56925,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
3,CNKSR2,22866,32.59,8.51,45.85,6.16,49.21,11.91,12.27,15.00,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
4,SCML1,6322,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,165.57,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,HAVCR2,84868,74.58,432.53,128.79,208.38,13.93,633.73,348.71,420.68,...,150.17,57.49,236.21,353.59,29.48,202.66,33.78,73.54,55.89,480.04
19644,RP1-66C13.4,0,0.00,0.00,0.00,0.00,0.00,0.00,7.69,0.00,...,0.00,31.00,1.83,0.00,3.32,5.73,0.00,3.03,0.00,0.00
19645,C3orf79,152118,2.27,1.66,0.00,1.22,0.00,0.00,0.00,0.00,...,0.00,5.50,0.00,0.00,0.00,2.86,0.00,1.04,1.50,0.00
19646,CTD-2116N17.1,0,6.89,10.79,8.51,6.84,9.13,9.93,14.78,15.56,...,8.85,6.21,4.94,17.64,28.65,14.78,27.64,15.80,31.00,4.86


In [33]:
#>>>>>>> 3 
# Data cleaning phase (1)
zero_index = []
for index in range(healthydf.shape[0]):
    if (healthydf.iloc[index][1:] == 0).all():
        zero_index.append(index)

healthy = healthydf.drop(healthydf.index[zero_index])
cancerous = cancerousdf.drop(cancerousdf.index[zero_index])
cancerous

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LIN7B,64130,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34
2,LXN,56925,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
3,CNKSR2,22866,32.59,8.51,45.85,6.16,49.21,11.91,12.27,15.00,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
4,SCML1,6322,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,165.57,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,HAVCR2,84868,74.58,432.53,128.79,208.38,13.93,633.73,348.71,420.68,...,150.17,57.49,236.21,353.59,29.48,202.66,33.78,73.54,55.89,480.04
19644,RP1-66C13.4,0,0.00,0.00,0.00,0.00,0.00,0.00,7.69,0.00,...,0.00,31.00,1.83,0.00,3.32,5.73,0.00,3.03,0.00,0.00
19645,C3orf79,152118,2.27,1.66,0.00,1.22,0.00,0.00,0.00,0.00,...,0.00,5.50,0.00,0.00,0.00,2.86,0.00,1.04,1.50,0.00
19646,CTD-2116N17.1,0,6.89,10.79,8.51,6.84,9.13,9.93,14.78,15.56,...,8.85,6.21,4.94,17.64,28.65,14.78,27.64,15.80,31.00,4.86


In [34]:
#>>>>>>> 4 
# Data cleaning phase (1)
num=0
drop_index = []
rows = healthy.shape[0]
for row in range (rows):
    x=healthy.iloc[row,2:]
    for index in x:
        if index ==0:
            num=num+1
    if num > 15:
        drop_index.append(row)
    num=0
healthy = healthy.drop(healthy.index[drop_index])
cancerous = cancerous.drop(cancerous.index[drop_index])
cancerous

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LIN7B,64130,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34
2,LXN,56925,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
3,CNKSR2,22866,32.59,8.51,45.85,6.16,49.21,11.91,12.27,15.00,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
4,SCML1,6322,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,165.57,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,ZNF521,25925,37.32,76.71,59.97,84.04,14.24,202.66,435.55,59.55,...,22.59,33.78,148.09,106.63,297.17,22.26,25.72,106.63,71.00,37.05
19642,SPINT2,10653,56265.94,3901.01,16843.62,12244.81,37901.36,8479.89,9606.86,10959.30,...,18052.61,15392.14,20030.74,20593.91,9808.75,11189.60,10733.74,10014.87,6983.79,23169.48
19643,HAVCR2,84868,74.58,432.53,128.79,208.38,13.93,633.73,348.71,420.68,...,150.17,57.49,236.21,353.59,29.48,202.66,33.78,73.54,55.89,480.04
19646,CTD-2116N17.1,0,6.89,10.79,8.51,6.84,9.13,9.93,14.78,15.56,...,8.85,6.21,4.94,17.64,28.65,14.78,27.64,15.80,31.00,4.86


In [35]:
#>>>>>>> 5
# GET DEGs (differentially expressed genes )
# Compare every row for float equality
# (rtol:Relative tolerance,atol:Absolute tolerance,equal_nan:If True, nans will be considered equal to nans)
common_index = []
close_options = {"rtol": 0.01, "atol": 0.01, "equal_nan": True}
for index in range(healthy.shape[0]):
    if np.allclose(np.float64(healthy.iloc[index][1:]),np.float64(cancerous.iloc[index][1:]), **close_options):
        common_index.append(index)

# Drop similarily-expressed genes
healthy = healthy.drop(healthy.index[common_index])
cancerous = cancerous.drop(cancerous.index[common_index])
cancerous

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,336.79,500.46,703.28,287.01,486.75,70.51,145.02,14.03,...,3.06,420.68,109.66,106.63,1233.75,172.65,303.44,228.13,251.48,23.59
1,LIN7B,64130,105.15,212.78,102.25,212.78,172.65,244.57,105.89,152.28,...,135.24,135.24,151.22,395.18,295.11,120.94,114.36,96.68,277.20,153.34
2,LXN,56925,848.22,236.21,271.48,759.08,61.25,620.67,329.84,599.49,...,688.78,204.07,438.59,503.95,3039.30,607.87,106.63,507.46,255.00,346.29
3,CNKSR2,22866,32.59,8.51,45.85,6.16,49.21,11.91,12.27,15.00,...,1.38,6.62,6.11,1.66,33.54,3.11,0.82,3.32,27.25,6.16
4,SCML1,6322,84.63,74.58,67.12,57.89,102.97,132.44,66.65,57.08,...,165.57,119.26,87.65,53.57,232.94,67.12,64.80,90.14,198.47,154.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,ZNF521,25925,37.32,76.71,59.97,84.04,14.24,202.66,435.55,59.55,...,22.59,33.78,148.09,106.63,297.17,22.26,25.72,106.63,71.00,37.05
19642,SPINT2,10653,56265.94,3901.01,16843.62,12244.81,37901.36,8479.89,9606.86,10959.30,...,18052.61,15392.14,20030.74,20593.91,9808.75,11189.60,10733.74,10014.87,6983.79,23169.48
19643,HAVCR2,84868,74.58,432.53,128.79,208.38,13.93,633.73,348.71,420.68,...,150.17,57.49,236.21,353.59,29.48,202.66,33.78,73.54,55.89,480.04
19646,CTD-2116N17.1,0,6.89,10.79,8.51,6.84,9.13,9.93,14.78,15.56,...,8.85,6.21,4.94,17.64,28.65,14.78,27.64,15.80,31.00,4.86


In [36]:
healthy

,Hugo_Symbol,Entrez_Gene_Id,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,LIN7B,64130,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,LXN,56925,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,CNKSR2,22866,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,SCML1,6322,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,ZNF521,25925,215.77,148.09,83.45,232.94,167.90,80.01,161.02,104.42,...,145.02,202.66,111.99,371.22,154.42,289.02,32.59,191.67,204.07,88.26
19642,SPINT2,10653,6792.79,5441.30,5831.91,5329.30,5711.87,6164.49,8134.41,6516.03,...,3955.48,4869.99,6935.54,4836.35,7230.10,6250.56,6840.04,5292.48,3928.15,7130.55
19643,HAVCR2,84868,423.61,529.06,660.68,620.67,518.15,848.22,366.09,1073.91,...,559.28,579.04,420.68,366.09,334.46,258.57,329.84,1030.12,665.29,633.73
19646,CTD-2116N17.1,0,0.00,0.00,0.99,3.59,2.23,3.35,0.00,7.34,...,6.94,1.00,0.00,4.58,0.96,3.06,1.83,1.77,3.63,2.56


In [37]:
#>>>>>>> 6
############################# 1.Test in case samples are paired (related) #####################################
paired=[]
rows = healthy.shape[0]
# EDIT HERE, i -> row
for row in range (rows):
    pv=ttest_rel(healthy.iloc[row,2:],cancerous.iloc[row,2:]).pvalue
    paired.append(pv)

In [38]:
#>>>>>>> 7
# FDR Correction 
corrected_paired=multipletests(paired, alpha=0.05, method='fdr_bh')[1]
corrected_paired

array([1.43531775e-07, 3.36108930e-01, 4.53718343e-04, ...,
       2.15161050e-12, 2.40612340e-10, 3.29367975e-06])

In [39]:
#>>>>>>> 8
# DEGs before and after the FDR correction in a data frame
significance_genes = pd.DataFrame({'Gene_name':healthy.iloc[:,0], 'p-values':paired, 'p-values_fdr':corrected_paired})
significance_genes

,Gene_name,p-values,p-values_fdr
0,HIST3H2A,4.043607e-08,1.435318e-07
1,LIN7B,2.891646e-01,3.361089e-01
2,LXN,2.322367e-04,4.537183e-04
3,CNKSR2,3.420577e-12,2.434863e-11
4,SCML1,6.251346e-02,8.331256e-02
...,...,...,...
19641,ZNF521,4.142164e-06,1.070816e-05
19642,SPINT2,2.452619e-07,7.685140e-07
19643,HAVCR2,2.435125e-13,2.151610e-12
19646,CTD-2116N17.1,4.129496e-11,2.406123e-10


In [40]:
#>>>>>>> 9
#Compare the two DEGs sets 
significance_genes['significance:p_vlaue'] = significance_genes['p-values'].apply(lambda x: x < 0.05)
significance_genes['significance:p_vlaue_fdr'] = significance_genes['p-values_fdr'].apply(lambda x: x < 0.05)
significance_genes


,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,4.043607e-08,1.435318e-07,True,True
1,LIN7B,2.891646e-01,3.361089e-01,False,False
2,LXN,2.322367e-04,4.537183e-04,True,True
3,CNKSR2,3.420577e-12,2.434863e-11,True,True
4,SCML1,6.251346e-02,8.331256e-02,False,False
...,...,...,...,...,...
19641,ZNF521,4.142164e-06,1.070816e-05,True,True
19642,SPINT2,2.452619e-07,7.685140e-07,True,True
19643,HAVCR2,2.435125e-13,2.151610e-12,True,True
19646,CTD-2116N17.1,4.129496e-11,2.406123e-10,True,True


In [41]:
#>>>>>>> 10
diffrentially_genes = significance_genes[significance_genes['significance:p_vlaue_fdr']== True]
diffrentially_genes

,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,4.043607e-08,1.435318e-07,True,True
2,LXN,2.322367e-04,4.537183e-04,True,True
3,CNKSR2,3.420577e-12,2.434863e-11,True,True
6,GSDMD,3.041721e-06,8.037167e-06,True,True
7,AKR1C1,1.938575e-05,4.499899e-05,True,True
...,...,...,...,...,...
19641,ZNF521,4.142164e-06,1.070816e-05,True,True
19642,SPINT2,2.452619e-07,7.685140e-07,True,True
19643,HAVCR2,2.435125e-13,2.151610e-12,True,True
19646,CTD-2116N17.1,4.129496e-11,2.406123e-10,True,True


In [49]:
#>>>>>>> 11
Gens=diffrentially_genes['Gene_name'].to_list()
len(Gens)
#print(Gens)

12393

In [43]:
#>>>>>>> 12
#################################### 2.Test in case samples are independent #############################################
independent=[]
rows = healthy.shape[0]
# EDIT HERE, i -> row
for row in range (rows):
    pv=ttest_ind(healthy.iloc[row,2:],cancerous.iloc[row,2:]).pvalue
    independent.append(pv)

In [44]:
#>>>>>>> 13
# FDR Correction 
corrected_ind=multipletests(independent, alpha=0.05, method='fdr_bh')[1]
corrected_ind

array([1.36094028e-08, 3.63452634e-01, 1.70306047e-04, ...,
       9.26766153e-14, 6.24144436e-12, 6.37027912e-07])

In [45]:
#>>>>>>> 14
# DEGs before and after the FDR correction in a data frame
significance_genes_ind = pd.DataFrame({'Gene_name':healthy.iloc[:,0], 'p-values':independent, 'p-values_fdr':corrected_ind})
significance_genes_ind

,Gene_name,p-values,p-values_fdr
0,HIST3H2A,3.607140e-09,1.360940e-08
1,LIN7B,3.138295e-01,3.634526e-01
2,LXN,8.164044e-05,1.703060e-04
3,CNKSR2,6.374652e-15,4.996380e-14
4,SCML1,4.726590e-02,6.481879e-02
...,...,...,...
19641,ZNF521,2.273493e-06,5.917339e-06
19642,SPINT2,5.250215e-08,1.699581e-07
19643,HAVCR2,1.228186e-14,9.267662e-14
19646,CTD-2116N17.1,1.068283e-12,6.241444e-12


In [46]:
#>>>>>>> 15
#Compare the two DEGs sets 
significance_genes_ind['significance:p_vlaue'] = significance_genes_ind['p-values'].apply(lambda x: x < 0.05)
significance_genes_ind['significance:p_vlaue_fdr'] = significance_genes_ind['p-values_fdr'].apply(lambda x: x < 0.05)
significance_genes_ind

,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,3.607140e-09,1.360940e-08,True,True
1,LIN7B,3.138295e-01,3.634526e-01,False,False
2,LXN,8.164044e-05,1.703060e-04,True,True
3,CNKSR2,6.374652e-15,4.996380e-14,True,True
4,SCML1,4.726590e-02,6.481879e-02,True,False
...,...,...,...,...,...
19641,ZNF521,2.273493e-06,5.917339e-06,True,True
19642,SPINT2,5.250215e-08,1.699581e-07,True,True
19643,HAVCR2,1.228186e-14,9.267662e-14,True,True
19646,CTD-2116N17.1,1.068283e-12,6.241444e-12,True,True


In [47]:
#>>>>>>> 16
#the common and distinct genes
diffrentially_indep = significance_genes_ind[significance_genes_ind['significance:p_vlaue_fdr']== True]
diffrentially_indep

,Gene_name,p-values,p-values_fdr,significance:p_vlaue,significance:p_vlaue_fdr
0,HIST3H2A,3.607140e-09,1.360940e-08,True,True
2,LXN,8.164044e-05,1.703060e-04,True,True
3,CNKSR2,6.374652e-15,4.996380e-14,True,True
6,GSDMD,5.344289e-06,1.321447e-05,True,True
7,AKR1C1,7.857877e-06,1.899042e-05,True,True
...,...,...,...,...,...
19641,ZNF521,2.273493e-06,5.917339e-06,True,True
19642,SPINT2,5.250215e-08,1.699581e-07,True,True
19643,HAVCR2,1.228186e-14,9.267662e-14,True,True
19646,CTD-2116N17.1,1.068283e-12,6.241444e-12,True,True


In [48]:
#>>>>>>> 17
Gens=diffrentially_indep['Gene_name'].to_list()
len(Gens)
#print(Gens)

12297